In [1]:
def make_helix_pairs(a_start, b_start, L):
    # Pair a_start+k with b_start+(L-1-k) => antiparallel helix
    return [(a_start + k, b_start + (L - 1 - k)) for k in range(L)]

def build_tetra_wireframe_blueprint(L=12, loop=4):
    """
    Tetrahedron graph has 6 edges:
      (0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
    For POC: each edge becomes one helix module: A(L) + loop + B(L) + loop
    Returns:
      N: total length
      pairs: list[(i,j)] base pairs (0-indexed)
      modules: list of dicts describing segments
    """
    edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
    pairs = []
    modules = []
    idx = 0

    for e in edges:
        a_start = idx; idx += L
        loop1_start = idx; idx += loop
        b_start = idx; idx += L
        loop2_start = idx; idx += loop

        pairs.extend(make_helix_pairs(a_start, b_start, L))
        modules.append({
            "edge": e,
            "A": (a_start, a_start+L-1),
            "loop1": (loop1_start, loop1_start+loop-1),
            "B": (b_start, b_start+L-1),
            "loop2": (loop2_start, loop2_start+loop-1),
        })

    return idx, pairs, modules

N, pairs, modules = build_tetra_wireframe_blueprint(L=12, loop=4)
print("Total nt:", N)
print("Total base pairs:", len(pairs))
print("First module:", modules[0])
print("First 8 pairs:", pairs[:8])

Total nt: 192
Total base pairs: 72
First module: {'edge': (0, 1), 'A': (0, 11), 'loop1': (12, 15), 'B': (16, 27), 'loop2': (28, 31)}
First 8 pairs: [(0, 27), (1, 26), (2, 25), (3, 24), (4, 23), (5, 22), (6, 21), (7, 20)]


In [2]:
def pairs_to_dotbracket(N, pairs):
    db = ['.'] * N
    used = set()
    for i, j in pairs:
        if i == j or i < 0 or j < 0 or i >= N or j >= N:
            raise ValueError("pair index out of range")
        if i in used or j in used:
            raise ValueError(f"position paired twice: {i},{j}")
        if i < j:
            db[i] = '('
            db[j] = ')'
        else:
            db[j] = '('
            db[i] = ')'
        used.add(i); used.add(j)
    return "".join(db)

db = pairs_to_dotbracket(N, pairs)
print("Dot-bracket head:", db[:120])
print("Dot-bracket tail:", db[-120:])
print("Unique chars:", set(db))


Dot-bracket head: ((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))
Dot-bracket tail: ((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))))))....
Unique chars: {')', '(', '.'}


In [3]:
import random

COMPS = {'A':'U','U':'A','G':'C','C':'G'}

def design_sequence_from_pairs(N, pairs, gc_bias=0.7, seed=0):
    random.seed(seed)
    seq = ['N'] * N
    pair_map = {}
    for i, j in pairs:
        pair_map[i] = j
        pair_map[j] = i

    for i, j in pairs:
        if i < j:
            if random.random() < gc_bias:
                left = random.choice(['G','C'])
                right = COMPS[left]
            else:
                left = random.choice(['A','U'])
                right = COMPS[left]
            seq[i] = left
            seq[j] = right

    # Fill unpaired positions (loops/junctions)
    for k in range(N):
        if seq[k] == 'N':
            seq[k] = random.choice(['A','U','G','C'])  # you can bias AU here if you want

    return "".join(seq)

seq = design_sequence_from_pairs(N, pairs, gc_bias=0.75, seed=42)
print("Sequence head:", seq[:120])


Sequence head: GGGGGGGCCAACUCUGGUUGGCCCCCCCCUUGGUCCGGUCUGGACGCCUCCAGACCGGACAUUAACCCCGGGCGGGGAUUCCCGCCCGGGGUAAAUUGUCCGCCUGGAAAGAUCCAGGCG


In [4]:
# !pip install RNA
!pip install ViennaRNA

In [5]:
import RNA

fc = RNA.fold_compound(seq)
mfe_struct, mfe = fc.mfe()

print("Target:", db[:120])
print("MFE   :", mfe_struct[:120])
print("MFE energy:", mfe)

# quick base-pair F1
def dotbracket_to_pairs(db_str):
    stack = []
    P = set()
    for i,ch in enumerate(db_str):
        if ch == '(':
            stack.append(i)
        elif ch == ')' and stack:
            j = stack.pop()
            P.add((j,i))
    return P

T = dotbracket_to_pairs(db)
P = dotbracket_to_pairs(mfe_struct)
tp = len(T & P); fp = len(P - T); fn = len(T - P)
prec = tp / max(tp+fp,1)
rec  = tp / max(tp+fn,1)
f1   = 2*prec*rec / max(prec+rec,1e-12)
print("PairF1 vs target:", f1)


ModuleNotFoundError: No module named 'RNA'

In [6]:
import sys, site
print("python:", sys.executable)
print("version:", sys.version)
print("site-packages:", site.getsitepackages())

python: /opt/miniconda3/envs/rna_genai/bin/python
version: 3.9.23 (main, Jun  5 2025, 08:49:36) 
[Clang 14.0.6 ]
site-packages: ['/opt/miniconda3/envs/rna_genai/lib/python3.9/site-packages']


In [7]:
import pkgutil, sys

mods = sorted([m.name for m in pkgutil.iter_modules()])
print("RNA" in mods, "ViennaRNA" in mods)

# Search anything that looks like vienna/rna
hits = [m for m in mods if "rna" in m.lower() or "vienna" in m.lower()]
print(hits[:50])


False True
['RNA3DB Parser Script', 'ViennaRNA', '_testinternalcapi', 'rna', 'tornado']


In [8]:
import importlib.util
print("spec RNA:", importlib.util.find_spec("RNA"))
print("spec ViennaRNA:", importlib.util.find_spec("ViennaRNA"))

spec RNA: None
spec ViennaRNA: ModuleSpec(name='ViennaRNA', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1036d3c40>, origin='/opt/miniconda3/envs/rna_genai/lib/python3.9/site-packages/ViennaRNA/__init__.py', submodule_search_locations=['/opt/miniconda3/envs/rna_genai/lib/python3.9/site-packages/ViennaRNA'])


In [9]:
import subprocess, shutil
print("RNAfold path:", shutil.which("RNAfold"))

RNAfold path: /opt/miniconda3/envs/rna_genai/bin/RNAfold


In [10]:
import subprocess

def rnafold(seq: str):
    p = subprocess.run(
        ["RNAfold", "--noPS"],
        input=(seq.strip() + "\n").encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        check=True
    )
    lines = p.stdout.decode().strip().splitlines()
    struct = lines[1].split()[0]
    energy = lines[1].split()[-1]  # e.g. "(-12.30)"
    return struct, energy


In [11]:
print(rnafold("GGGAAACCC"))


('(((...)))', '-1.20)')


In [12]:
def dotbracket_to_pairs(db_str):
    stack = []
    P = set()
    for i,ch in enumerate(db_str):
        if ch == '(':
            stack.append(i)
        elif ch == ')' and stack:
            j = stack.pop()
            P.add((j,i))
    return P

T_pairs = dotbracket_to_pairs(db)

def pair_f1(pred_db, true_pairs=T_pairs):
    P_pairs = dotbracket_to_pairs(pred_db)
    tp = len(true_pairs & P_pairs)
    fp = len(P_pairs - true_pairs)
    fn = len(true_pairs - P_pairs)
    prec = tp / max(tp+fp, 1)
    rec  = tp / max(tp+fn, 1)
    f1   = 2*prec*rec / max(prec+rec, 1e-12)
    return prec, rec, f1


In [13]:
best = None
NUM_TRIES = 300

for seed in range(NUM_TRIES):
    cand = design_sequence_from_pairs(N, pairs, gc_bias=0.85, seed=seed)
    pred_db, energy = rnafold(cand)
    p, r, f1 = pair_f1(pred_db)

    if (best is None) or (f1 > best["f1"]):
        best = {"seed": seed, "seq": cand, "pred_db": pred_db, "energy": energy, "p": p, "r": r, "f1": f1}

print("BEST PairF1:", best["f1"], "| P:", best["p"], "R:", best["r"], "| Energy:", best["energy"], "| seed:", best["seed"])
print("\nTarget (head):", db[:120])
print("Pred   (head):", best["pred_db"][:120])
print("Seq    (head):", best["seq"][:120])


BEST PairF1: 1.0 | P: 1.0 R: 1.0 | Energy: (-163.40) | seed: 3

Target (head): ((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))
Pred   (head): ((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))))))....((((((((((((....))))))))
Seq    (head): GCGCGCCGGUGGGUGUCCACCGGCGCGCGUCAGCCCCAACCCCCAGGUGGGGGUUGGGGCCUAGCGACAGCGCGGCUCGUGCCGCGCUGUCGAAUGGGCCGGGUCGUGUGGACACGACCC


In [14]:
import numpy as np

# --- Choose geometry scale ---
# Approx A-form rise per bp ~ 2.8 Å (0.28 nm). This is a ballpark.
RISE_A = 2.8  # Angstrom per base step along helix axis
STRAND_OFFSET = 2.0  # Å separation between the two strands in the helix (visual only)

# Edges in your tetra blueprint (should match your build function)
edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]

def tetra_vertices(edge_len=12*RISE_A):
    """
    Returns 4 vertices of a tetrahedron centered near origin.
    edge_len controls size (Å).
    """
    # Regular tetrahedron coordinates (normalized), then scaled
    V = np.array([
        [ 1,  1,  1],
        [ 1, -1, -1],
        [-1,  1, -1],
        [-1, -1,  1],
    ], dtype=float)
    # Scale so that edge lengths match edge_len
    # Current edge length between V[0] and V[1] is sqrt((0)^2+(2)^2+(2)^2)=sqrt(8)=2*sqrt(2)
    curr = np.linalg.norm(V[0] - V[1])
    V *= (edge_len / curr)
    return V

def unit(v):
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def orthonormal_perp(axis):
    """
    Get a stable perpendicular unit vector to 'axis' for offsetting strands.
    """
    axis = unit(axis)
    # pick a reference vector not parallel to axis
    ref = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(axis, ref)) > 0.9:
        ref = np.array([0.0, 1.0, 0.0])
    perp = np.cross(axis, ref)
    perp = unit(perp)
    return perp

def write_pdb(points, out_path="wireframe_tetra.pdb"):
    """
    points: list of dicts with keys:
      - idx (int): nucleotide index
      - xyz (np.array shape (3,))
      - chain (str)
      - resi (int)
      - name (str) atom name
    """
    lines = []
    atom_id = 1
    for p in points:
        x, y, z = p["xyz"]
        # PDB columns are fixed-width; keep it simple
        lines.append(
            f"ATOM  {atom_id:5d} {p['name']:<4s} RNA {p['chain']}{p['resi']:4d}    "
            f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           P"
        )
        atom_id += 1
    lines.append("END")
    with open(out_path, "w") as f:
        f.write("\n".join(lines))
    return out_path

def build_tetra_wireframe_points(L=12, loop=4, rise=RISE_A, offset=STRAND_OFFSET):
    """
    Creates a PDB-like point model for the tetrahedron wireframe.
    For each edge:
      - strand A points go from vertex u -> v
      - strand B points go from vertex v -> u (reverse)
      Both are offset sideways for visual separation.
    """
    V = tetra_vertices(edge_len=L * rise)  # make edge length match helix axis length
    points = []

    # Build per-edge points, independent of your sequence indexing.
    # We'll label residues sequentially per edge for easy viewing.
    resi = 1
    chain = "A"

    for e_idx, (u, v) in enumerate(edges):
        p0 = V[u]
        p1 = V[v]
        axis = p1 - p0
        axis_u = unit(axis)
        perp = orthonormal_perp(axis_u)

        # Strand A: u -> v
        for k in range(L):
            t = (k + 0.5) / L
            xyz = (1 - t) * p0 + t * p1 + perp * (offset / 2.0)
            points.append({"idx": None, "xyz": xyz, "chain": chain, "resi": resi, "name": "P"})
            resi += 1

        # Loop spacer (unpaired) near vertex v (just place near end)
        for k in range(loop):
            xyz = p1 + perp * (offset * 0.8) + axis_u * (k - loop/2) * 0.8
            points.append({"idx": None, "xyz": xyz, "chain": chain, "resi": resi, "name": "P"})
            resi += 1

        # Strand B: v -> u (reverse direction)
        for k in range(L):
            t = (k + 0.5) / L
            xyz = (1 - t) * p1 + t * p0 - perp * (offset / 2.0)
            points.append({"idx": None, "xyz": xyz, "chain": chain, "resi": resi, "name": "P"})
            resi += 1

        # Loop spacer near vertex u
        for k in range(loop):
            xyz = p0 - perp * (offset * 0.8) - axis_u * (k - loop/2) * 0.8
            points.append({"idx": None, "xyz": xyz, "chain": chain, "resi": resi, "name": "P"})
            resi += 1

    return points

# ---- Generate and write PDB ----
# Use the same L and loop you used in your blueprint (you can change these)
L = 12
loop = 4

points = build_tetra_wireframe_points(L=L, loop=loop)
pdb_path = write_pdb(points, out_path="wireframe_tetra.pdb")
print("Wrote:", pdb_path)
print("Total points:", len(points))


Wrote: wireframe_tetra.pdb
Total points: 192


In [15]:
!pip install rna-tools

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 3.8 MB/s  0:00:15m0:00:0100:01
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  DEPRECATION: Building 'csvsort' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'csvsort'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for csvsort: filename=csvsort-1.6.1-py3-none-any.whl size=4130 sha256=00abc006e49cb5b3179959734ce5ca3a9495db92764a5ad82f9e65540c24a258
  Stored in directory: /Users/eshitagupta/Library/Caches/pip/wheels/b2/65/94/49f6e02d1e5d47833da

In [ ]:
import numpy as np

# -----------------------------
# Geometry params (A-form-ish)
# -----------------------------
RISE = 2.8          # Å per nucleotide step along helix axis (rough)
TWIST_DEG = 32.7    # degrees per step (rough A-form-ish)
R_P = 8.5           # Å helix radius for phosphates (visual)
R_S = 6.0           # Å radius for sugar (C4') (visual)
BASE_R = 3.5        # Å radial placement for "base" pseudo atoms (visual)

# -----------------------------
# Helpers
# -----------------------------
def unit(v):
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def rot_from_z_to_vec(v):
    """
    Rotation matrix R so that R * [0,0,1] = unit(v)
    """
    v = unit(v)
    z = np.array([0.0, 0.0, 1.0])

    if np.allclose(v, z):
        return np.eye(3)
    if np.allclose(v, -z):
        # 180 deg rotation around x
        return np.array([[1,0,0],[0,-1,0],[0,0,-1]], dtype=float)

    axis = unit(np.cross(z, v))
    angle = np.arccos(np.clip(np.dot(z, v), -1.0, 1.0))

    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ], dtype=float)

    R = np.eye(3) + np.sin(angle)*K + (1 - np.cos(angle))*(K @ K)
    return R

def pdb_atom_line(atom_id, atom_name, res_name, chain_id, res_id, xyz, element="C"):
    x, y, z = xyz
    # PDB fixed columns (kept simple but readable by ChimeraX)
    return (
        f"ATOM  {atom_id:5d} {atom_name:<4s}{res_name:>3s} {chain_id}{res_id:4d}    "
        f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           {element:>2s}"
    )

# -----------------------------
# Idealized A-form duplex generator (local Z-axis helix)
# Produces two anti-parallel strands A and B
# -----------------------------
def build_aform_duplex_local(L, seqA=None, seqB=None, start_res=1):
    """
    Returns list of atoms dicts for a duplex along +Z axis:
      Strand A: residues 1..L
      Strand B: residues 1..L but placed anti-parallel (reverse along Z)

    Each residue has pseudo atoms:
      P, C4', N (a single base atom: N9 for purines, N1 for pyrimidines)
    """
    if seqA is None:
        seqA = ("G" * L)
    if seqB is None:
        # naive complement-ish: G<->C, A<->U
        comp = {"A":"U","U":"A","G":"C","C":"G"}
        seqB = "".join(comp.get(b, "C") for b in seqA)

    assert len(seqA) == L and len(seqB) == L

    atoms = []

    twist = np.deg2rad(TWIST_DEG)

    # Strand A along +Z
    for i in range(L):
        base = seqA[i]
        theta = i * twist
        z = (i + 0.5) * RISE

        # Phosphate
        p = np.array([R_P*np.cos(theta), R_P*np.sin(theta), z])
        # Sugar
        c4 = np.array([R_S*np.cos(theta + 0.3), R_S*np.sin(theta + 0.3), z - 0.3])
        # Base pseudo atom points inward toward helix center
        inward = unit(-np.array([np.cos(theta), np.sin(theta), 0.0]))
        n_atom = c4 + inward * BASE_R

        resname = base  # ChimeraX understands A/U/G/C residue names commonly
        base_atom_name = "N9" if base in ("A","G") else "N1"

        atoms.append(("A", start_res + i, "P",  p,  "P"))
        atoms.append(("A", start_res + i, "C4'", c4, "C"))
        atoms.append(("A", start_res + i, base_atom_name, n_atom, "N"))

    # Strand B anti-parallel: we run residues 1..L but map to reverse z positions
    # Place opposite side (theta + pi) and reverse z (L-1-i)
    for i in range(L):
        base = seqB[i]
        theta = (L - 1 - i) * twist + np.pi
        z = (i + 0.5) * RISE

        p = np.array([R_P*np.cos(theta), R_P*np.sin(theta), z])
        c4 = np.array([R_S*np.cos(theta + 0.3), R_S*np.sin(theta + 0.3), z - 0.3])
        inward = unit(-np.array([np.cos(theta), np.sin(theta), 0.0]))
        n_atom = c4 + inward * BASE_R

        resname = base
        base_atom_name = "N9" if base in ("A","G") else "N1"

        atoms.append(("B", start_res + i, "P",  p,  "P"))
        atoms.append(("B", start_res + i, "C4'", c4, "C"))
        atoms.append(("B", start_res + i, base_atom_name, n_atom, "N"))

    return atoms

# -----------------------------
# Build tetrahedron vertices
# -----------------------------
def tetra_vertices(edge_len_angstrom):
    """
    Regular tetrahedron centered near origin, scaled to edge_len_angstrom.
    """
    V = np.array([
        [ 1,  1,  1],
        [ 1, -1, -1],
        [-1,  1, -1],
        [-1, -1,  1],
    ], dtype=float)
    curr = np.linalg.norm(V[0] - V[1])  # = 2*sqrt(2)
    V *= (edge_len_angstrom / curr)
    return V

# -----------------------------
# Assemble a tetra wireframe:
# each edge = one duplex oriented along that edge
# chains: per edge we assign unique chain IDs for clarity
# -----------------------------
def build_tetra_wireframe_pdb(
    L=12,
    out_path="tetra_aform_wireframe.pdb",
    loop=0,
):
    # Edges of tetrahedron
    edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]

    edge_len = L * RISE  # match helix length to edge length
    V = tetra_vertices(edge_len_angstrom=edge_len)

    # chain IDs (ChimeraX colors by chain nicely)
    chain_ids = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789")
    assert len(edges)*2 <= len(chain_ids), "Not enough chain IDs."

    lines = []
    atom_id = 1

    for e_idx, (u, v) in enumerate(edges):
        p0 = V[u]
        p1 = V[v]
        axis = p1 - p0
        R = rot_from_z_to_vec(axis)

        # local helix runs from z ~ [0, L*RISE]
        # translate so helix center starts near p0 and ends near p1
        # We'll place local z=0 at p0
        T = p0

        # pick a simple sequence (GC rich) per edge for visual stability
        seqA = ("GCGC" * ((L//4)+1))[:L]
        comp = {"A":"U","U":"A","G":"C","C":"G"}
        seqB = "".join(comp[b] for b in seqA)

        atoms_local = build_aform_duplex_local(L=L, seqA=seqA, seqB=seqB, start_res=1)

        # Assign unique chains for each duplex strand per edge
        chainA = chain_ids[2*e_idx]
        chainB = chain_ids[2*e_idx + 1]

        for (strand, resi, aname, xyz, element) in atoms_local:
            xyz_world = (R @ xyz) + T

            if strand == "A":
                chain = chainA
            else:
                chain = chainB

            # residue name: use the actual base letter for nicer display
            # (ChimeraX recognizes standard nucleotides)
            # We'll infer from atom name presence in local list by residue index:
            # easiest: just set resname = "A" for now? Better: keep as "A"/"U"/"G"/"C" is fine.
            # We can pick from seqA/seqB:
            if strand == "A":
                resname = seqA[resi-1]
            else:
                resname = seqB[resi-1]

            lines.append(pdb_atom_line(atom_id, aname, resname, chain, resi, xyz_world, element=element))
            atom_id += 1

        # TER record separates chains cleanly
        lines.append(f"TER   {atom_id:5d}")
        atom_id += 1

    lines.append("END")

    with open(out_path, "w") as f:
        f.write("\n".join(lines))

    print(f"Wrote {out_path}")
    print(f"Edges: {len(edges)} | Duplex strands: {len(edges)*2} | L={L}")

# ---- Run it ----
build_tetra_wireframe_pdb(L=12, out_path="tetra_aform_wireframe.pdb")


Wrote: tetra_aform_wireframe_bonded.pdb


In [18]:
import numpy as np

# -----------------------------
# Geometry params (A-form-ish)
# -----------------------------
RISE = 2.8          # Å per nucleotide step along helix axis (rough)
TWIST_DEG = 32.7    # degrees per step (rough A-form-ish)
R_P = 8.5           # Å helix radius for phosphates (visual)
R_S = 6.0           # Å radius for sugar (C4') (visual)
BASE_R = 3.5        # Å radial placement for "base" pseudo atoms (visual)

# -----------------------------
# Helpers
# -----------------------------
def unit(v):
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def rot_from_z_to_vec(v):
    v = unit(v)
    z = np.array([0.0, 0.0, 1.0])

    if np.allclose(v, z):
        return np.eye(3)
    if np.allclose(v, -z):
        return np.array([[1,0,0],[0,-1,0],[0,0,-1]], dtype=float)

    axis = unit(np.cross(z, v))
    angle = np.arccos(np.clip(np.dot(z, v), -1.0, 1.0))

    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ], dtype=float)

    R = np.eye(3) + np.sin(angle)*K + (1 - np.cos(angle))*(K @ K)
    return R

def pdb_atom_line(atom_id, atom_name, res_name, chain_id, res_id, xyz, element="C"):
    x, y, z = xyz
    return (
        f"ATOM  {atom_id:5d} {atom_name:<4s}{res_name:>3s} {chain_id}{res_id:4d}    "
        f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           {element:>2s}"
    )

def write_conect(bonds):
    """
    bonds: dict[int, set[int]] adjacency, atom serial numbers 1-based
    PDB CONECT lines allow up to 4 partners per line (we can emit multiple lines).
    """
    lines = []
    for a in sorted(bonds.keys()):
        nbrs = sorted(bonds[a])
        # split into chunks of 4
        for i in range(0, len(nbrs), 4):
            chunk = nbrs[i:i+4]
            lines.append(f"CONECT{a:5d}" + "".join(f"{b:5d}" for b in chunk))
    return lines

# -----------------------------
# Idealized A-form duplex generator (local Z-axis helix)
# Adds bond topology info (per-residue and along backbone)
# -----------------------------
def build_aform_duplex_local_with_bonds(L, seqA=None, seqB=None, start_res=1):
    if seqA is None:
        seqA = ("G" * L)
    if seqB is None:
        comp = {"A":"U","U":"A","G":"C","C":"G"}
        seqB = "".join(comp.get(b, "C") for b in seqA)

    assert len(seqA) == L and len(seqB) == L

    # atoms list with placeholders for atom serial assignment later
    atoms = []
    # for bonding, keep per-strand per-residue references to atom indices in `atoms`
    # We'll store indices into atoms list; later map to serial numbers
    ref = {"A": [], "B": []}

    twist = np.deg2rad(TWIST_DEG)

    def add_residue(strand, resi, base, theta, z):
        # Positions
        p  = np.array([R_P*np.cos(theta),       R_P*np.sin(theta),       z])
        c4 = np.array([R_S*np.cos(theta + 0.3), R_S*np.sin(theta + 0.3), z - 0.3])
        inward = unit(-np.array([np.cos(theta), np.sin(theta), 0.0]))
        n_atom = c4 + inward * BASE_R

        base_atom_name = "N9" if base in ("A","G") else "N1"

        iP  = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":"P",   "xyz":p,     "elem":"P"})
        iC4 = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":"C4'", "xyz":c4,    "elem":"C"})
        iN  = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":base_atom_name, "xyz":n_atom, "elem":"N"})

        ref[strand].append({"P": iP, "C4": iC4, "N": iN})

    # Strand A along +Z
    for i in range(L):
        base = seqA[i]
        theta = i * twist
        z = (i + 0.5) * RISE
        add_residue("A", start_res + i, base, theta, z)

    # Strand B anti-parallel
    for i in range(L):
        base = seqB[i]
        theta = (L - 1 - i) * twist + np.pi
        z = (i + 0.5) * RISE
        add_residue("B", start_res + i, base, theta, z)

    # Bonds adjacency (by atoms-list index for now)
    bonds_idx = {i: set() for i in range(len(atoms))}

    def bond(i, j):
        bonds_idx[i].add(j)
        bonds_idx[j].add(i)

    # Intra-residue bonds: P—C4'—N
    for strand in ("A", "B"):
        for r in ref[strand]:
            bond(r["P"], r["C4"])
            bond(r["C4"], r["N"])

    # Backbone bonds: connect P(i)—P(i+1) for each strand (simple)
    for strand in ("A", "B"):
        for k in range(L-1):
            bond(ref[strand][k]["P"], ref[strand][k+1]["P"])

    # Base-pair bonds (visual ladder): N_A(i) — N_B(L-1-i)
    for i in range(L):
        j = L - 1 - i
        bond(ref["A"][i]["N"], ref["B"][j]["N"])

    return atoms, bonds_idx, ref

# -----------------------------
# Tetrahedron vertices
# -----------------------------
def tetra_vertices(edge_len_angstrom):
    V = np.array([
        [ 1,  1,  1],
        [ 1, -1, -1],
        [-1,  1, -1],
        [-1, -1,  1],
    ], dtype=float)
    curr = np.linalg.norm(V[0] - V[1])  # = 2*sqrt(2)
    V *= (edge_len_angstrom / curr)
    return V

# -----------------------------
# Assemble tetra wireframe and write PDB with CONECT
# -----------------------------
def build_tetra_wireframe_pdb_bonded(L=12, out_path="tetra_aform_wireframe_bonded.pdb"):
    edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
    edge_len = L * RISE * 1.6
    V = tetra_vertices(edge_len_angstrom=edge_len)

    chain_ids = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789")
    assert len(edges)*2 <= len(chain_ids)

    # global atom list and global bonds on serial numbers
    pdb_lines = []
    global_bonds = {}  # serial -> set(serial)
    atom_id = 1

    def add_global_bond(a, b):
        global_bonds.setdefault(a, set()).add(b)
        global_bonds.setdefault(b, set()).add(a)

    for e_idx, (u, v) in enumerate(edges):
        p0 = V[u]
        p1 = V[v]
        R = rot_from_z_to_vec(p1 - p0)
        T = p0

        # GC-rich edge for stability/visual
        seqA = ("GCGC" * ((L//4)+1))[:L]
        comp = {"A":"U","U":"A","G":"C","C":"G"}
        seqB = "".join(comp[b] for b in seqA)

        atoms_local, bonds_idx, _ = build_aform_duplex_local_with_bonds(L=L, seqA=seqA, seqB=seqB, start_res=1)

        chainA = chain_ids[2*e_idx]
        chainB = chain_ids[2*e_idx + 1]

        # Map local atom index -> global serial number
        local_to_serial = {}

        # Write atoms
        for i, a in enumerate(atoms_local):
            xyz_world = (R @ a["xyz"]) + T
            chain = chainA if a["strand"] == "A" else chainB
            pdb_lines.append(pdb_atom_line(atom_id, a["aname"], a["resname"], chain, a["resi"], xyz_world, element=a["elem"]))
            local_to_serial[i] = atom_id
            atom_id += 1

        # Convert local bonds to global bonds
        for i in bonds_idx:
            si = local_to_serial[i]
            for j in bonds_idx[i]:
                sj = local_to_serial[j]
                if sj != si:
                    add_global_bond(si, sj)

        pdb_lines.append(f"TER   {atom_id:5d}")
        atom_id += 1

    # Add CONECT lines
    conect_lines = write_conect(global_bonds)

    with open(out_path, "w") as f:
        f.write("\n".join(pdb_lines + conect_lines + ["END"]))

    print("Wrote:", out_path)

# ---- Run it ----
build_tetra_wireframe_pdb_bonded(L=12)


Wrote: tetra_aform_wireframe_bonded.pdb


In [19]:
import numpy as np
import subprocess

# -----------------------------
# A-form-ish parameters (visual)
# -----------------------------
RISE = 2.8          # Å per nucleotide step along helix axis (rough)
TWIST_DEG = 32.7    # deg per step
R_P = 8.5           # Å phosphate radius
R_S = 6.0           # Å sugar radius
BASE_R = 3.5        # Å base pseudo atom offset inward

# Junction params
JUNCTION_GAP = 10.0     # Å: how far to stop helices short of the vertex
JUNCTION_ARM_LEN = 6    # residues in each connector arm (visual)
JUNCTION_RADIUS = 7.0   # Å: hub ring radius (visual)

# -----------------------------
# Helpers
# -----------------------------
def unit(v):
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def rot_from_z_to_vec(v):
    v = unit(v)
    z = np.array([0.0, 0.0, 1.0])

    if np.allclose(v, z):
        return np.eye(3)
    if np.allclose(v, -z):
        return np.array([[1,0,0],[0,-1,0],[0,0,-1]], dtype=float)

    axis = unit(np.cross(z, v))
    angle = np.arccos(np.clip(np.dot(z, v), -1.0, 1.0))

    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ], dtype=float)

    R = np.eye(3) + np.sin(angle)*K + (1 - np.cos(angle))*(K @ K)
    return R

def pdb_atom_line(atom_id, atom_name, res_name, chain_id, res_id, xyz, element="C"):
    x, y, z = xyz
    return (
        f"ATOM  {atom_id:5d} {atom_name:<4s}{res_name:>3s} {chain_id}{res_id:4d}    "
        f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           {element:>2s}"
    )

def write_conect(bonds):
    lines = []
    for a in sorted(bonds.keys()):
        nbrs = sorted(bonds[a])
        for i in range(0, len(nbrs), 4):
            chunk = nbrs[i:i+4]
            lines.append(f"CONECT{a:5d}" + "".join(f"{b:5d}" for b in chunk))
    return lines

# -----------------------------
# Local duplex along +Z with bonds
# -----------------------------
def build_aform_duplex_local_with_bonds(L, seqA, seqB, start_res=1):
    atoms = []
    ref = {"A": [], "B": []}
    twist = np.deg2rad(TWIST_DEG)

    def add_residue(strand, resi, base, theta, z):
        p  = np.array([R_P*np.cos(theta),       R_P*np.sin(theta),       z])
        c4 = np.array([R_S*np.cos(theta + 0.3), R_S*np.sin(theta + 0.3), z - 0.3])
        inward = unit(-np.array([np.cos(theta), np.sin(theta), 0.0]))
        n_atom = c4 + inward * BASE_R

        base_atom_name = "N9" if base in ("A","G") else "N1"
        iP  = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":"P",   "xyz":p,     "elem":"P"})
        iC4 = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":"C4'", "xyz":c4,    "elem":"C"})
        iN  = len(atoms); atoms.append({"strand": strand, "resi": resi, "resname": base, "aname":base_atom_name, "xyz":n_atom, "elem":"N"})
        ref[strand].append({"P": iP, "C4": iC4, "N": iN})

    # Strand A
    for i in range(L):
        theta = i * twist
        z = (i + 0.5) * RISE
        add_residue("A", start_res + i, seqA[i], theta, z)

    # Strand B anti-parallel
    for i in range(L):
        theta = (L - 1 - i) * twist + np.pi
        z = (i + 0.5) * RISE
        add_residue("B", start_res + i, seqB[i], theta, z)

    bonds_idx = {i: set() for i in range(len(atoms))}
    def bond(i, j):
        bonds_idx[i].add(j)
        bonds_idx[j].add(i)

    # Intra-residue: P—C4'—N
    for strand in ("A", "B"):
        for r in ref[strand]:
            bond(r["P"], r["C4"])
            bond(r["C4"], r["N"])

    # Backbone: P(i)—P(i+1)
    for strand in ("A", "B"):
        for k in range(L-1):
            bond(ref[strand][k]["P"], ref[strand][k+1]["P"])

    # Base-pair ladder: N_A(i)—N_B(L-1-i)
    for i in range(L):
        j = L - 1 - i
        bond(ref["A"][i]["N"], ref["B"][j]["N"])

    return atoms, bonds_idx, ref

# -----------------------------
# Tetrahedron vertices & edges
# -----------------------------
def tetra_vertices(edge_len_angstrom):
    V = np.array([
        [ 1,  1,  1],
        [ 1, -1, -1],
        [-1,  1, -1],
        [-1, -1,  1],
    ], dtype=float)
    curr = np.linalg.norm(V[0] - V[1])  # 2*sqrt(2)
    V *= (edge_len_angstrom / curr)
    return V

edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]

# -----------------------------
# Junction builder: hub + 3 arms (sticks)
# -----------------------------
def build_junction(vertex_xyz, arm_targets, chain_id, start_res, atom_id_start):
    """
    vertex_xyz: np.array(3,)
    arm_targets: list of 3 np.array(3,) points (helix endpoints)
    Returns:
      pdb_lines, bonds(serial adjacency), next_atom_id, next_res_id,
      arm_end_serials (serial of last P on each arm to bond to helix endpoint)
    """
    pdb_lines = []
    bonds = {}
    atom_id = atom_id_start
    res_id = start_res

    def add_bond(a,b):
        bonds.setdefault(a,set()).add(b)
        bonds.setdefault(b,set()).add(a)

    # Create 3 hub residues in a small ring around vertex
    # (purely to make a visible junction “node”)
    # Choose arbitrary plane based on first two targets
    a0 = unit(arm_targets[0] - vertex_xyz)
    a1 = unit(arm_targets[1] - vertex_xyz)
    n = unit(np.cross(a0, a1))
    if np.linalg.norm(n) < 1e-6:
        n = np.array([0,0,1.0])
    t = unit(np.cross(n, a0))

    hub_pts = []
    for k in range(3):
        ang = 2*np.pi*k/3
        hub_pts.append(vertex_xyz + JUNCTION_RADIUS*(np.cos(ang)*a0 + np.sin(ang)*t))

    hub_P_serials = []
    # Each hub residue: P, C4', N (U)
    for k in range(3):
        P = hub_pts[k]
        C4 = P + 1.5*unit(vertex_xyz - P)
        Np = C4 + 2.0*unit(vertex_xyz - C4)

        pdb_lines.append(pdb_atom_line(atom_id, "P", "U", chain_id, res_id, P, element="P")); sP = atom_id; atom_id += 1
        pdb_lines.append(pdb_atom_line(atom_id, "C4'", "U", chain_id, res_id, C4, element="C")); sC4 = atom_id; atom_id += 1
        pdb_lines.append(pdb_atom_line(atom_id, "N1", "U", chain_id, res_id, Np, element="N")); sN = atom_id; atom_id += 1

        add_bond(sP, sC4)
        add_bond(sC4, sN)

        hub_P_serials.append(sP)
        res_id += 1

    # Connect hub ring P-P
    add_bond(hub_P_serials[0], hub_P_serials[1])
    add_bond(hub_P_serials[1], hub_P_serials[2])
    add_bond(hub_P_serials[2], hub_P_serials[0])

    # Now 3 arms: from hub residue k P -> helix endpoint target
    arm_end_serials = []
    for k, target in enumerate(arm_targets):
        startP = hub_pts[k]
        prevP = hub_P_serials[k]

        for step in range(JUNCTION_ARM_LEN):
            frac = (step + 1) / (JUNCTION_ARM_LEN + 1)
            P = (1-frac)*startP + frac*target
            C4 = P + 1.2*unit(vertex_xyz - P)
            Np = C4 + 1.8*unit(vertex_xyz - C4)

            pdb_lines.append(pdb_atom_line(atom_id, "P", "U", chain_id, res_id, P, element="P")); sP = atom_id; atom_id += 1
            pdb_lines.append(pdb_atom_line(atom_id, "C4'", "U", chain_id, res_id, C4, element="C")); sC4 = atom_id; atom_id += 1
            pdb_lines.append(pdb_atom_line(atom_id, "N1", "U", chain_id, res_id, Np, element="N")); sN = atom_id; atom_id += 1

            add_bond(sP, sC4)
            add_bond(sC4, sN)
            add_bond(prevP, sP)  # backbone-ish
            prevP = sP
            res_id += 1

        arm_end_serials.append(prevP)

    return pdb_lines, bonds, atom_id, res_id, arm_end_serials

# -----------------------------
# Main: build tetra wireframe with junction hubs
# -----------------------------
def build_tetra_with_junctions(L=12, out_path="tetra_wireframe_with_junctions.pdb"):
    # Make tetra size ~ helix length + gaps for junctions
    edge_len = L * RISE + 2*JUNCTION_GAP
    V = tetra_vertices(edge_len_angstrom=edge_len)

    chain_ids = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789")
    atom_id = 1
    res_id = 1

    pdb_lines = []
    global_bonds = {}

    def add_global_bond(a,b):
        global_bonds.setdefault(a,set()).add(b)
        global_bonds.setdefault(b,set()).add(a)

    # Store helix endpoint phosphate serials to connect to junction arms
    # key: (vertex_index, edge_index, strand_letter, which_end) -> serial
    # which_end: 0 = near u, 1 = near v
    helix_endP = {}

    # Build edge helices (shortened so they stop before vertex)
    for e_idx, (u, v) in enumerate(edges):
        p0 = V[u]
        p1 = V[v]
        axis = p1 - p0
        dir_uv = unit(axis)

        # helix endpoints stop short of vertices
        start = p0 + dir_uv * JUNCTION_GAP
        end   = p1 - dir_uv * JUNCTION_GAP

        # local helix along +Z with length L*RISE. Place it between start->end
        R = rot_from_z_to_vec(end - start)
        T = start

        # GC-rich sequence per edge
        seqA = ("GCGC" * ((L//4)+1))[:L]
        comp = {"A":"U","U":"A","G":"C","C":"G"}
        seqB = "".join(comp[b] for b in seqA)

        atoms_local, bonds_idx, ref = build_aform_duplex_local_with_bonds(L=L, seqA=seqA, seqB=seqB, start_res=1)

        chainA = chain_ids[2*e_idx]
        chainB = chain_ids[2*e_idx + 1]

        local_to_serial = {}
        for i, a in enumerate(atoms_local):
            xyz_world = (R @ a["xyz"]) + T
            chain = chainA if a["strand"] == "A" else chainB
            pdb_lines.append(pdb_atom_line(atom_id, a["aname"], a["resname"], chain, a["resi"], xyz_world, element=a["elem"]))
            local_to_serial[i] = atom_id
            atom_id += 1

        # bonds
        for i in bonds_idx:
            si = local_to_serial[i]
            for j in bonds_idx[i]:
                sj = local_to_serial[j]
                if sj != si:
                    add_global_bond(si, sj)

        # record endpoint phosphate serials (strand A only, both ends) for junction connections
        # "near u" is residue 1 P; "near v" is residue L P in local Z direction
        end_u_A = local_to_serial[ref["A"][0]["P"]]
        end_v_A = local_to_serial[ref["A"][L-1]["P"]]

        helix_endP[(u, e_idx, "A", 0)] = end_u_A
        helix_endP[(v, e_idx, "A", 1)] = end_v_A

        pdb_lines.append(f"TER   {atom_id:5d}")
        atom_id += 1

    # Build junctions at each vertex: connect to the 3 incident edges (strand A only)
    for vtx in range(4):
        incident = []
        for e_idx, (u, w) in enumerate(edges):
            if u == vtx or w == vtx:
                incident.append((e_idx, u, w))
        # tetrahedron: exactly 3 incident edges
        assert len(incident) == 3

        arm_targets = []
        helix_serials_to_connect = []

        for (e_idx, u, w) in incident:
            # get helix endpoint P serial at this vertex
            if u == vtx:
                sP = helix_endP[(u, e_idx, "A", 0)]
            else:
                sP = helix_endP[(w, e_idx, "A", 1)]
            helix_serials_to_connect.append(sP)

            # approximate 3D target point for junction arm: use helix P coordinate is not stored,
            # so we connect by bond only; geometry-wise, arm endpoints are built toward vertex.
            # We'll build arms toward the vertex itself for clean look:
            arm_targets.append(V[vtx])

        # Use a dedicated chain per junction so it's easy to color
        chainJ = chain_ids[12 + vtx]  # after helix chains

        j_lines, j_bonds, atom_id, res_id, arm_end_serials = build_junction(
            vertex_xyz=V[vtx],
            arm_targets=arm_targets,
            chain_id=chainJ,
            start_res=res_id,
            atom_id_start=atom_id
        )

        pdb_lines.extend(j_lines)
        for a in j_bonds:
            for b in j_bonds[a]:
                add_global_bond(a, b)

        # Connect each arm end to a helix endpoint phosphate by bond (visual)
        for arm_end, helix_end in zip(arm_end_serials, helix_serials_to_connect):
            add_global_bond(arm_end, helix_end)

        pdb_lines.append(f"TER   {atom_id:5d}")
        atom_id += 1

    conect_lines = write_conect(global_bonds)
    with open(out_path, "w") as f:
        f.write("\n".join(pdb_lines + conect_lines + ["END"]))

    print("Wrote:", out_path)

# ---- Run ----
build_tetra_with_junctions(L=12, out_path="tetra_wireframe_with_junctions.pdb")


Wrote: tetra_wireframe_with_junctions.pdb
